In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
import os

print(os.listdir(r'C:\Users\ADMIN\Downloads'))

['.ipynb_checkpoints', 'AD3491-Fundamentals-of-Data-Science-and-Analytics-Lecture-Notes-1.pdf', 'AIDS Repository - NM Project.xlsx', 'Awesome Van Gogh Paintings Hd Download.jfif', 'Canva Setup 1.110.0.exe', 'Computer Networks - CS3591 - Notes 4.pdf', 'Computer Networks - CS3591 - Notes.pdf', 'COMPUTER NETWORKS.CN.pdf', 'CPU-Scheduling-and-Algorithm-.pdf', 'CS3591 - CN.docx', 'CS3591-Computer-Networks-Lecture-Notes-1 (1).pdf', 'CS3591-Computer-Networks-Lecture-Notes-1.pdf', 'CS3591-Computer-Networks-Lecture-Notes-2.pdf', 'Dataset .csv', 'desktop.ini', 'diabetes.csv', 'DOC-20250501-WA0003..pdf', 'Document.odt', 'DS_Record_Final.pdf', 'Fundamentals-of-Data-Science-and-Analytics-Apr-May-2024-Question-Paper-Download.pdf', 'Git-2.49.0-64-bit.exe', 'GitHubDesktopSetup-x64.exe', 'Green Beige  Minimalist Ebook Leadmagnet Workbook.pdf', 'jdk-21_windows-x64_bin.exe', 'MA3391-Probability-and-Statistics-Apr-May-2024-Question-Paper-Download.pdf', 'Machine Learning Internship Task  (1).pdf', 'Machine

In [3]:
import pandas as pd

df = pd.read_csv(r'C:\Users\ADMIN\Downloads\Dataset .csv')
print(df.head())

   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 

In [4]:
# Drop rows with missing 'Cuisines'
df_cleaned = df.dropna(subset=['Cuisines']).copy()

# Create a new column combining 'Cuisines' and 'Average Cost for two'
df_cleaned['combined_features'] = df_cleaned['Cuisines'] + " " + df_cleaned['Average Cost for two'].astype(str)


In [5]:
# Create a TF-IDF Vectorizer object
vectorizer = TfidfVectorizer()

# Fit and transform the 'combined_features' column
feature_vectors = vectorizer.fit_transform(df_cleaned['combined_features'])


In [6]:
from sklearn.preprocessing import LabelEncoder

# Create a dictionary to store all encoders
label_encoders = {}

# List of categorical columns to encode
categorical_cols = ['City', 'Has Table booking', 'Has Online delivery']

# Loop through and encode each column
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Convert text to numeric
    label_encoders[col] = le  # Save encoder for future decoding



In [7]:
def recommend_restaurants(user_cuisine, user_budget, top_n=5):
    # Step 1: Create a dummy user input like "Japanese 500"
    user_input = user_cuisine + " " + str(user_budget)
    
    # Step 2: Convert user input into a vector
    user_vector = vectorizer.transform([user_input])
    
    # Step 3: Calculate similarity with all restaurants
    similarity_scores = cosine_similarity(user_vector, feature_vectors).flatten()
    
    # Step 4: Get top matching restaurant indices
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    # Step 5: Return recommended restaurants
    return df_cleaned.iloc[top_indices][['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Aggregate rating']]


In [8]:
# Example: Recommend Japanese restaurants within a budget of ₹500
recommendations = recommend_restaurants("Japanese", 500)

# Show the result
print(recommendations)


                 Restaurant Name           Cuisines  Average Cost for two  \
29                      New Koto           Japanese                   200   
54                  Sushi Leblon           Japanese                   250   
1517                    Daikichi           Japanese                  1000   
5417  Manami Japanese Restaurant           Japanese                  1000   
6400             Dimsum Vs Sushi  Japanese, Chinese                   800   

      Aggregate rating  
29                 3.7  
54                 4.6  
1517               3.6  
5417               0.0  
6400               3.4  
